# Lab 3: LLM-Based Agents with tools

## Introduction

In this lab, you'll build a chatbot that represents you professionally — capable of answering questions based on your LinkedIn and bio, and that notifies you via mobile push messages using Pushover.

You'll learn to:

- Use Pushover for notifications
- Set up OpenAI's tool-calling API
- Load your own data (e.g., résumé) to simulate a personalized assistant
- Deploy the chatbot with Gradio
- Handle unknown questions or user interest via tools

### 1. Setup and Imports

In [ ]:
!pip install pypdf gradio

In [ ]:
# imports

from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr

1. **Visit Gemini Developer Page:**

  Go to: https://ai.google.dev/

  Click “Get started” and log in using your Google account.

2. **Generate Your API Key:**

  Visit: https://aistudio.google.com/app/apikey

  Click “Create API Key”.

  Copy the generated key.

In [ ]:
# The usual start

google_api_key = '' #add api key
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.0-flash"


### 2. Setup Pushover Notifications

Pushover is a service to push messages to your phone or desktop.

It's super easy to set up and install!

Simply visit https://pushover.net/ and click 'Login or Signup' on the top right to sign up for a free account, and create your API keys.

Once you've signed up, on the home screen, click "Create an Application/API Token", and give it any name (like Agents) and click Create Application.

Then add 2 keys:

PUSHOVER_USER=_put the key that's on the top right of your Pushover home screen and probably starts with a u_  
PUSHOVER_TOKEN=_put the key when you click into your new application called Agents (or whatever) and probably starts with an a_

Finally, click "Add Phone, Tablet or Desktop" to install on your phone.

In [ ]:
# For pushover

pushover_user = '' # add key here
pushover_token = '' # add key here
pushover_url = "https://api.pushover.net/1/messages.json"

In [ ]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [ ]:
push("HEY!!")

Push: HEY!!


### 3. Tool Definitions

We define tool functions the LLM can use via tool calling.



In [ ]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

In [ ]:
def record_unknown_question(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

JSON Schemas for the Tools

In [ ]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [ ]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [ ]:
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

In [ ]:
tools

[{'type': 'function',
  'function': {'name': 'record_user_details',
   'description': 'Use this tool to record that a user is interested in being in touch and provided an email address',
   'parameters': {'type': 'object',
    'properties': {'email': {'type': 'string',
      'description': 'The email address of this user'},
     'name': {'type': 'string',
      'description': "The user's name, if they provided it"},
     'notes': {'type': 'string',
      'description': "Any additional information about the conversation that's worth recording to give context"}},
    'required': ['email'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_question',
   'description': "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': "The question that couldn't be answered"}},
    'required': ['quest

### 4. Handling Tool Calls (LLM responses)

In [ ]:
# This function can take a list of tool calls, and run them. This is the IF statement!!

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)

        # THE BIG IF STATEMENT!!!

        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)

        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [ ]:
globals()["record_unknown_question"]("this is a really hard question")

Push: Recording this is a really hard question asked that I couldn't answer


{'recorded': 'ok'}

In [ ]:
# This is a more elegant way that avoids the IF statement.

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

### 5. Load LinkedIn and Summary

Your chatbot needs to know about you in order to behave like your professional digital twin. In this step, you will provide it with two key documents:

**1. LinkedIn Profile (PDF)** :

How to export your LinkedIn profile:
1. Go to linkedin.com.
2. Click “Me” (top right) → View Profile.
3. Click the “Resources” button near your profile picture.
4. Select “Save to PDF”.

A PDF will download — rename it as linkedin.pdf and set the path.
If you dont have linkedin you can just create one about yourself.

**2. Professional Summary (Text File)** :

Write a short summary of your:
- Career goals
- Skills and interests
- Education
- What makes you unique


In [ ]:
reader = PdfReader("Profile.pdf") #linkedin path
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("summary.txt", "r", encoding="utf-8") as f:  # summary path
    summary = f.read()

name = " " #your name

### 6. System Prompt for the Chatbot
This defines how the agent should behave.

In [ ]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


### 7. Chat Function (with Tool Support)
This function powers the chatbot and handles responses + tool cal

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:

        # This is the call to the LLM - see that we pass in the tools json

        response = gemini.chat.completions.create(model=model_name, messages=messages,tools=tools)

        finish_reason = response.choices[0].finish_reason

        # If the LLM wants to call a tool, we do that!

        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content

### 9. Launch with Gradio UI

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

### Conclusion
You've built a custom personal chatbot that:

- Reads your resume
- Answers career-related questions
- Pushes notifications to your phone
- Records user interest and unknown queries using OpenAI tool calling